In [1]:
from IPython.display import clear_output
from kendama_env import KendamaEnv
import numpy as np 
import time
env = KendamaEnv()

a = 0
start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time

    if elapsed_time > 1.0/240:
        a += 0.005
        start_time = time.time()
        # action = [list(np.random.normal(scale = 0,loc=1,size = 3)),list(np.random.normal(scale = 10,size = 3))]
        action = [0,0,0.*np.sin(20*a),0,0,0]
        
        clear_output(wait=True)
        
        ob, reward, done, _ = env.step(action)
        
        print(ob)
        # pas de pv
        #if(done):
        #    env.reset()

error: Not connected to physics server.

import gym
from IPython.display import clear_output
from kendama_env import KendamaEnv
import numpy as np 
import time
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from stable_baselines.common.vec_env import VecNormalize

env = KendamaEnv(render=True)
env = DummyVecEnv([lambda:env])

model = PPO2(MlpPolicy, env, verbose=0,tensorboard_log="./log_model/", gamma=0.99)
#model.learn(total_timesteps=10000000)

In [2]:

model.learn(total_timesteps=2000000,tb_log_name="vitesse4.0", reset_num_timesteps=True)

ValueError: could not broadcast input array from shape (10,3) into shape (4,3)

In [1]:
%load_ext tensorboard
%tensorboard --logdir ./log_model/


Reusing TensorBoard on port 6006 (pid 25000), started 10:01:40 ago. (Use '!kill 25000' to kill it.)

In [ ]:
obs = env.reset()
start_time = time.time()
while 1:
    current_time = time.time()
    elapsed_time = current_time - start_time
    if elapsed_time > 1.0/20:
        start_time=current_time
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        if dones:
            obs = env.reset()
